In [0]:
INPUT_FILE_NAME = 'log_sample.csv'
#'small_log.csv'
#'log_sample.csv'
LABEL_TIME_PART = 0.2

In [0]:
import csv
import numpy as np
from tqdm import tqdm
import operator

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!unzip "drive/My Drive/log_sample.zip" -d "."

Archive:  drive/My Drive/log_sample.zip
  inflating: ./log_sample.csv        


In [0]:
class Event:
    def __init__(self, timestamp, group_id, device_id, event_id, event_count):
        self.timestamp = int(timestamp)
        self.group_id = group_id
        self.event_id = event_id
        self.device_id = device_id
        if event_count:
            self.count = int(event_count)
        else:
            self.count = 0

In [0]:
def check_group_event_id(group_id, event_id):
    return group_id not in ('performance', 'vcs.change.reminder') and event_id not in ('ui.lagging',
            'ide.error',
            'ide.freeze',
            'ui.latency',
            'registered','invoked',
            'TsLintLanguageService',
            'whitelist.updated',
            'logs.send',
            'notification.shown',
            'ESLintLanguageService'
            )

In [0]:
def read_from_file_to_event_list():
    events = []
    isFirst = True
    with open(INPUT_FILE_NAME, 'r') as fin:
        for row in tqdm(csv.reader(fin, delimiter=',')):
            if isFirst:
                isFirst = False
                continue
            if (row[12].split('.')[0]) and check_group_event_id(row[5], row[10]):
                events.append(Event(row[3], row[5], row[7], row[10], row[12].split('.')[0]))
    return events        
            

In [10]:
events = read_from_file_to_event_list()

44349155it [06:12, 119116.97it/s]


In [0]:
def get_user_to_time_period(events):
    user_to_min_time = {}
    user_to_max_time = {}
    labels = {}

    for event in tqdm(events):
        if event.device_id not in user_to_min_time.keys():
            user_to_min_time[event.device_id] = event.timestamp
        else:
            user_to_min_time[event.device_id] = min(user_to_min_time[event.device_id], event.timestamp)
        if event.device_id not in user_to_max_time.keys():
            user_to_max_time[event.device_id] = event.timestamp
        else:
            user_to_max_time[event.device_id] = max(user_to_max_time[event.device_id], event.timestamp)

    user_to_time_period = {}
    
    for user in user_to_min_time.keys():
        min_time = user_to_min_time[user]
        max_time = user_to_max_time[user]
        user_to_time_period[user] = int((max_time - min_time) / 1000 /60/60/24)
    return user_to_time_period  
        
    

In [17]:
user_to_time_period = get_user_to_time_period(events).values()


100%|██████████| 20551899/20551899 [00:24<00:00, 843811.06it/s]

In [18]:
print(len(user_to_time_period))

3711


In [0]:
period_to_count = np.zeros(100)
for period in user_to_time_period:
    period_to_count[period] += 1


In [21]:
for i in range(len(period_to_count)):
    print(str(i) + " days: " + str(int(period_to_count[i])) + " users")

0 days: 1062 users
1 days: 207 users
2 days: 124 users
3 days: 117 users
4 days: 80 users
5 days: 85 users
6 days: 78 users
7 days: 75 users
8 days: 62 users
9 days: 47 users
10 days: 54 users
11 days: 39 users
12 days: 35 users
13 days: 34 users
14 days: 35 users
15 days: 41 users
16 days: 30 users
17 days: 32 users
18 days: 26 users
19 days: 22 users
20 days: 44 users
21 days: 31 users
22 days: 28 users
23 days: 35 users
24 days: 30 users
25 days: 22 users
26 days: 18 users
27 days: 25 users
28 days: 39 users
29 days: 15 users
30 days: 21 users
31 days: 18 users
32 days: 10 users
33 days: 17 users
34 days: 25 users
35 days: 28 users
36 days: 16 users
37 days: 17 users
38 days: 27 users
39 days: 23 users
40 days: 22 users
41 days: 19 users
42 days: 30 users
43 days: 22 users
44 days: 17 users
45 days: 26 users
46 days: 22 users
47 days: 12 users
48 days: 17 users
49 days: 27 users
50 days: 19 users
51 days: 9 users
52 days: 19 users
53 days: 18 users
54 days: 13 users
55 days: 12 user

In [0]:
def get_user_to_time_periods(events):
    user_to_timestamps = {}

    for event in tqdm(events):
        if event.device_id in user_to_timestamps.keys():
            user_to_timestamps[event.device_id].append(event.timestamp)
        else:
            user_to_timestamps[event.device_id] = [event.timestamp]
      
    user_to_time_periods = {}
    for user in tqdm(user_to_timestamps.keys()):
        user_to_timestamps[user] = sorted(user_to_timestamps[user])
        user_to_time_periods[user] = []
        for i in range(len(user_to_timestamps[user]) - 1):
            diff = int((user_to_timestamps[user][i + 1] - user_to_timestamps[user][i]) / 1000 / 60)
            if diff > 0:
                user_to_time_periods[user].append(diff)
    return user_to_time_periods
    

In [23]:
user_to_time_periods = get_user_to_time_periods(events)


100%|██████████| 3711/3711 [00:14<00:00, 261.11it/s] 

In [24]:
activity = []
for value in user_to_time_periods.values():
    cnt = 0
    for elem in value:
        if elem <= 30:
            cnt += elem
        else:
            #print("active(" + str(cnt), end=') ')
            activity.append(cnt)
            cnt = 0
            #print("stopped(" + str(elem), end=') ')
            #print()
    #print("active(" + str(cnt), end=') ')
    activity.append(cnt)  
#print(activity)
x = np.percentile(activity, 95)
print(x)

x1 = np.mean(activity)
print(x1)

x2 = np.median(activity)
print(x2)
    #print("END")

107.0
27.28991852671015
12.0


In [0]:
import sys

In [0]:
def get_user_to_action_cnt(events):
    user_to_cnt = {}
    for event in tqdm(events):
        if event.device_id not in user_to_cnt.keys():
            user_to_cnt[event.device_id] = sys.getsizeof(event.timestamp) + sys.getsizeof(event.group_id) + sys.getsizeof(event.event_id)  
        else:
            user_to_cnt[event.device_id] += sys.getsizeof(event.timestamp) + sys.getsizeof(event.group_id) + sys.getsizeof(event.event_id)   
    return max(user_to_cnt.items(), key=operator.itemgetter(1))

In [15]:
get_user_to_action_cnt(events)

100%|██████████| 20551899/20551899 [00:30<00:00, 669014.86it/s]


('2903191c0a437cb-8223-49d2-bdc3-ef36e49b8389', 44679581)

In [16]:
44679581 / 1024 / 1024

42.609768867492676